In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from google.colab import drive
from tqdm import tqdm
import dill
import numpy as np
import random
import copy

In [ ]:
drive.mount('/content/drive')

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/embeddings_dict.pkl', 'rb') as pkl:
    embeddings_dict = dill.load(pkl)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/ai_word_bank_list.pkl', 'rb') as pkl:
    ai_word_bank_list = dill.load(pkl)

In [ ]:
with open('/content/drive/My Drive/Chameleon_AI/Pickles/topic_dict.pkl', 'rb') as pkl:
    topic_dict = dill.load(pkl)

In [ ]:
#added later for further development
with open('/content/drive/My Drive/Chameleon_AI/Pickles/similarity_dict.pkl', 'rb') as pkl:
    similarity_dict = dill.load(pkl)

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"

In [ ]:
USE_model = hub.load(module_url)

In [ ]:
def most_similar(phrase: str) -> str :
    phrase_embedding = USE_model([phrase])
    #current_word_embedding = ""
    most_similar_value = 0
    most_similar_string = ""
    for word in ai_word_bank_list:
        if word == phrase:
            continue
        if np.inner(phrase_embedding, embeddings_dict[word]) > most_similar_value:
            most_similar_value = np.inner(phrase_embedding, embeddings_dict[word])
            most_similar_string = word
    return most_similar_string, most_similar_value

In [ ]:
most_similar("smoothie")

In [ ]:
def build_similarity_dict (topic_dict: dict) -> dict :
    similarity_dict = {}
    for topic in topic_dict:
        similarity_dict[topic] = {}
        for word in tqdm(ai_word_bank_list):
            if topic == word:
                continue
            else:
                #print(topic)
                similarity_dict[topic][word] = np.inner(embeddings_dict[word],
                                                        embeddings_dict[topic])
        for secret_word in topic_dict[topic]:
            similarity_dict[secret_word] = {}
            for element in tqdm(ai_word_bank_list):
                if secret_word == word:
                    continue
                else:
                    similarity_dict[secret_word][element] = \
                    np.inner(embeddings_dict[secret_word],
                             embeddings_dict[element])
    return similarity_dict

In [ ]:
similarity_dict = build_similarity_dict(topic_dict)

In [ ]:
#with open('/content/drive/My Drive/Chameleon_AI/Pickles/similarity_dict.pkl', 'wb') as pkl:
#    dill.dump(similarity_dict, pkl)

In [ ]:
#this won't make a difference again if this gets adressed earlier in the pipline
#delete_small_words_list = []
#for word in ai_word_bank_list:
#    if len(word) <= 2:
#        delete_small_words_list.append(word)

In [ ]:
#this won't make a difference again if this gets adressed earlier in the pipline
#for word in delete_small_words_list:
#    if word in ai_word_bank_list:
#        ai_word_bank_list.remove(word)

In [ ]:
#with open('/content/drive/My Drive/Chameleon_AI/Pickles/ai_word_bank_list.pkl', 'wb') as pkl:
#    dill.dump(ai_word_bank_list, pkl)

In [ ]:
#for word in delete_small_words_list:
#    embeddings_dict.pop(word)

In [ ]:
#with open('/content/drive/My Drive/Chameleon_AI/Pickles/embeddings_dict.pkl', 'wb') as pkl:
#    dill.dump(embeddings_dict, pkl)

In [ ]:
def ai_player_guess(phrase: str) -> dict :
    phrase = phrase.lower()
    guessing_dict = {}
    dict_element_counter = 0
    phrase_embedding = USE_model([phrase])
    #current_word_embedding = ""
    minimum_similar_value = 0
    minimum_similar_string = ""
    for word in ai_word_bank_list:
        if word == phrase or phrase[:3] in word:
            continue
        if dict_element_counter < 100:
            dict_element_counter += 1
            guessing_dict[word] = np.inner(phrase_embedding, embeddings_dict[word])  
            minimum_similar_string = min(guessing_dict, key=guessing_dict.get)
            #print(minimum_similar_string)
            #print(smoothie_dict[word])
            minimum_similar_value = guessing_dict[minimum_similar_string]            
        
        elif np.inner(phrase_embedding, embeddings_dict[word]) > minimum_similar_value:
            guessing_dict.pop(minimum_similar_string)
            guessing_dict[word] = np.inner(phrase_embedding, embeddings_dict[word])
            minimum_similar_string = min(guessing_dict, key=guessing_dict.get)
            minimum_similar_value = guessing_dict[minimum_similar_string]
    
    sorted_similar_words = sorted(guessing_dict.items(), key=lambda x: x[1],
                                  reverse=True)

    return sorted_similar_words
    
    #for i in sorted_similar_words:
	    #print(i[0], float(i[1]))

In [ ]:
def score_calibrator(phrase: str):
    high_list = []
    low_list = []
    fine_list = []
    sorted_similar_words = ai_player_guess(phrase)
    for i in sorted_similar_words:
        choice =input(f"Is *{i[0]}* with a score of {i[1]} too (h)ighly related,"\
              f" (l)lowly related, or just (f)ine?")
        if choice == 'h':
            high_list.append(i)
            print(f"You added {i[0]} with {i[1]} as high similarity.")
        elif choice == 'l':
            low_list.append(i)
            print(f"You added {i[0]} with {i[1]} as low similarity.")
        elif choice == 'f':
            print(f"You added {i[0]} with {i[1]} as fine similarity.")
            fine_list.append(i)
        else:
            print(f"Error. {choice} is an invalid command.")
    return high_list, low_list, fine_list

In [ ]:
high_list, low_list, fine_list = score_calibrator("Steam Engine")

In [ ]:
def string_checker(string1, string2):
    string1 = string1.lower()
    string2= string2.lower()
    if len(string1) >= 3:
        if string1[:3] in string2:
            return True
    if len(string2) >= 3:
        if string2[:3] in string1:
            return True
    if len(string1) >= 4 and len(string2) >= 4:
        for index, letter in enumerate(string1):
            if index <= (len(string1) - 4):
                sub_string = string1[index : index + 4]
                if sub_string in string2:
                    return True
    if len(string2) >= 4 and len(string1) >=4:
        for index, letter in enumerate(string2):
            if index <= (len(string2) - 4):
                sub_string = string2[index : index +4]
                if sub_string in string1:
                    return True
    return False

In [ ]:
string_checker('Cruise Ship', 'ships')

In [ ]:
if string_checker('gun', 'gunfight'):
    print('balls')

In [ ]:
def randomize_secret_word():
    random_topic = random.choice(list(topic_dict))
    topic_word_spread = topic_dict[random_topic]
    secret_word = random.choice(topic_word_spread)
    topic_word_spread.remove(secret_word)
    return random_topic, topic_word_spread, secret_word

In [ ]:
the_topic, non_secret_words, the_secret_word = randomize_secret_word()
print(f"The topic is {the_topic}.")
print(f"The non-secret words in this topic are {non_secret_words}.")
print(f"The secret word is {the_secret_word}.")

In [ ]:
def ai_player_hint():
    topic, non_secret_words, secret_word = randomize_secret_word()
    ai_available_words = copy.deepcopy(ai_word_bank_list)
    try:
        ai_available_words.remove(topic.lower())
    except ValueError:
        pass

    for word in non_secret_words:
        try:
            ai_available_words.remove(word.lower())
        except ValueError:
            continue

    print(secret_word)
    still_search = True
    
    hint_candidate = hint_search(secret_word, topic,
                                 non_secret_words, ai_available_words)
    
    print(f"hint for {secret_word} is {hint_candidate} at " \
          f"{similarity_dict[secret_word][hint_candidate]}")


    return hint_candidate

In [ ]:
true_hint = ai_player_hint()

In [ ]:
def hint_search(secret_word, topic, non_secret_words, ai_available_words):
    while len(ai_available_words) >= 1:
        print(len(ai_available_words))
        hint_candidate = (random.choice(ai_available_words))
        if string_checker(secret_word, hint_candidate):
            ai_available_words.remove(hint_candidate)
            continue
        if similarity_dict[secret_word][hint_candidate] >= .48 and \
            similarity_dict[secret_word][hint_candidate] <= .52:
            print(f"secret {secret_word} matches with {hint_candidate} at " \
                  f"{similarity_dict[secret_word][hint_candidate]}.")
            if string_checker(hint_candidate, topic):
                ai_available_words.remove(hint_candidate)
                continue
            if check_topic(hint_candidate, topic):
                if check_non_secret_words(hint_candidate, non_secret_words):
                    return hint_candidate
                else:
                    ai_available_words.remove(hint_candidate)
                    continue            
            else:
                ai_available_words.remove(hint_candidate)
                continue
        else:
            print(f"Fail: secret {secret_word} no match with {hint_candidate} at " \
                  f"{similarity_dict[secret_word][hint_candidate]}.")
            ai_available_words.remove(hint_candidate)
            #print('lets try again')
            #print(hint_candidate)
            #print(similarity_dict[secret_word][hint_candidate])

In [ ]:
def check_topic(hint_candidate, topic):
    if similarity_dict[topic][hint_candidate] >= .2 and \
    similarity_dict[topic][hint_candidate] <= .785:
        print(f"{topic} topic match at{similarity_dict[topic][hint_candidate]}")
        return True
    else:
        print(f"{topic} failure did not match at " \
              f"{similarity_dict[topic][hint_candidate]}")
        return False

In [ ]:
def check_non_secret_words(hint_candidate, non_secret_words):
    non_secret_options = copy.deepcopy(non_secret_words)
    non_secret_success_count = 0
    match_list = []
    while len(non_secret_options) >= 1:
        non_secret_word = random.choice(non_secret_options)
        if string_checker(hint_candidate, non_secret_word):
            return False
        if similarity_dict[non_secret_word][hint_candidate] >= .41 and \
        similarity_dict[non_secret_word][hint_candidate] <= .785:
            non_secret_success_count +=1
            match_list.append(non_secret_word)
            non_secret_options.remove(non_secret_word)
        else:
            non_secret_options.remove(non_secret_word)
    
    if non_secret_success_count >= 2:
        print(f"Success: {hint_candidate} matched with " \
              f"{non_secret_success_count} words.")
        print(match_list)
        return True
    else:
        print(f"Failure: {hint_candidate} matched with " \
              f"{non_secret_success_count} words.")
        return False

In [ ]:
def go_through_all_options():
    for topic in topic_dict:
        topic_word_spread = topic_dict[topic]
        for secret_word in topic_dict[topic]:
            ai_player_hint_limit(topic, non_secret_words, secret_word)

In [ ]:
def randomize_secret_word():
    random_topic = random.choice(list(topic_dict))
    topic_word_spread = topic_dict[random_topic]
    secret_word = random.choice(topic_word_spread)
    topic_word_spread.remove(secret_word)
    return random_topic, topic_word_spread, secret_word

In [ ]:
scope_variable = 0

In [ ]:
def test_func_a():
    scope_variable = 5

In [ ]:
def test_func_b():
    scope_variable = 10    

In [ ]:
print(scope_variable)

In [ ]:
similarity_dict['Coca-Cola']['coke']